In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import datetime

import numpy as np
import pandas as pd
import json
import time


In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('')

In [4]:
info = client.get_all_tickers()

In [5]:
quote_ccy = 'USDT'

In [6]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [7]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [8]:
target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 300 and d['rank'] <= 350]

In [9]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [10]:
target_ticker_list

['FETUSDT',
 'STPTUSDT',
 'SUNUSDT',
 'FLMUSDT',
 'UTKUSDT',
 'XVSUSDT',
 'ALPHAUSDT',
 'CTKUSDT',
 'RIFUSDT',
 'ALICEUSDT',
 'PERPUSDT',
 'SUPERUSDT',
 'TLMUSDT',
 'BAKEUSDT',
 'XVGUSDT',
 'GHSTUSDT',
 'TRIBEUSDT',
 'BETAUSDT',
 'DARUSDT',
 'RGTUSDT',
 'MOVRUSDT',
 'ACHUSDT']

In [11]:
#target_ticker_list = ['MBLUSDT']

In [12]:
def is_candidate(filtered_klines_info, tolerable_volume): 
    trigger_kline = filtered_klines_info[-1]
    
    drop_from_high_gate = trigger_kline[2]/trigger_kline[4] < 1.06
    
    trigger_gate = 15 > trigger_kline[-1] > 10 
    

    if trigger_gate and True:
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [13]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[4] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [14]:
trade_list = []
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "30 day ago UTC") 
    length = int(len(klines))
    if(length < 24):
        continue
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)

    for x in range(0, length):
        
        hit_ticker_dictionary = {}
        
        filtered_klines_info = focused_kline_array[x:x+12]
        
        if(len(filtered_klines_info) == 12):
            context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
            average_volume = np.mean(context_klines_info[:,5])
            tolerable_volume = average_volume
        
        
            buy_kline_info = filtered_klines_info[-1]


            if(is_candidate(filtered_klines_info, tolerable_volume)):

                hit_ticker_dictionary = {}
                buy_price = buy_kline_info[4]
                hit_ticker_dictionary['ticker'] = target_ticker
                hit_ticker_dictionary['buy_price'] = buy_kline_info[4]

                buy_timesstamp = buy_kline_info[0]
                hit_ticker_dictionary['buy_datetime'] = str(datetime.fromtimestamp(buy_timesstamp // 1000))
                
                proft_percentage = 1.10
                loss_percentage = 0.75
                profit_sell_price = buy_price * proft_percentage
                loss_sell_price = buy_price * loss_percentage
                
                
                
                usdt = 1000
                
                buy_quantity = usdt/buy_price
                hit_ticker_dictionary['quantity'] = buy_quantity
                
                for y in range(x+12, length):
                    high_price = focused_kline_array[y-1:y][0][2]
                    low_price = focused_kline_array[y-1:y][0][3]
                    close_price = focused_kline_array[y-1:y][0][4]
                    sell_timesstamp = focused_kline_array[y-1:y][0][0]
                    sell_datetime = str(datetime.fromtimestamp(sell_timesstamp // 1000))


                    if high_price >= profit_sell_price:
                        hit_ticker_dictionary['sell_price'] = profit_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = profit_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * proft_percentage - usdt
                        
                        break
                        
                    elif low_price <= loss_sell_price and False:
                        hit_ticker_dictionary['sell_price'] = loss_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = loss_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * loss_percentage - usdt
                        break
                    elif sell_timesstamp - buy_timesstamp >= 14400000 :
                        hit_ticker_dictionary['sell_price'] = close_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = close_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = buy_quantity * close_price - buy_quantity * buy_price
                        break
                        
                        
                        
                        
                        
                trade_list.append(hit_ticker_dictionary)
                
                busd = 100

                quantity = busd/ buy_kline_info[4]
                quantity_str = "{:0.0{}f}".format(quantity, 2)

                """
                order = client.create_test_order(symbol=target_ticker, side=SIDE_BUY, type='MARKET', quoteOrderQty=busd)
                time.sleep(5)
                order_id = order['orderId'] 
                '''order_id = client.get_all_orders(symbol=target_ticker)[0]['orderId'] '''

                buy_price = order['fills'][0]['price']
                buy_qty = order['executedQty']
                sell_price = buy_price * 1.03



                while client.get_order(symbol=target_ticker, orderId=order_id)['status'] != 'FILLED':
                    time.sleep(10)

                if client.get_order(symbol=target_ticker, orderId=order_id)['status'] == 'FILLED':
                    asset = target_ticker.split("USDT")[0]
                    balance = client.get_asset_balance(asset=asset)
                    free_to_trade = balance['free']
                    sell_price_str = "{:0.0{}f}".format(sell_price, 2)

                    order = client.create_test_order(symbol=target_ticker, side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)
                    print('********** SELL ORDER SET ************************')

                    """


Analyzing Hourly Started 
FETUSDT
Analyzing Hourly Started 
STPTUSDT
Analyzing Hourly Started 
SUNUSDT
Analyzing Hourly Started 
FLMUSDT
Analyzing Hourly Started 
UTKUSDT
Analyzing Hourly Started 
XVSUSDT
Analyzing Hourly Started 
ALPHAUSDT
Analyzing Hourly Started 
CTKUSDT
Analyzing Hourly Started 
RIFUSDT
Analyzing Hourly Started 
ALICEUSDT
Analyzing Hourly Started 
PERPUSDT
Analyzing Hourly Started 
SUPERUSDT
Analyzing Hourly Started 
TLMUSDT
Analyzing Hourly Started 
BAKEUSDT
Analyzing Hourly Started 
XVGUSDT
Analyzing Hourly Started 
GHSTUSDT
Analyzing Hourly Started 
TRIBEUSDT
Analyzing Hourly Started 
BETAUSDT
Analyzing Hourly Started 
DARUSDT
Analyzing Hourly Started 
RGTUSDT
Analyzing Hourly Started 
MOVRUSDT
Analyzing Hourly Started 
ACHUSDT


In [15]:
trade_list

[{'ticker': 'FLMUSDT',
  'buy_price': 0.1373,
  'buy_datetime': '2022-06-20 13:00:00',
  'quantity': 7283.3211944646755,
  'sell_price': 0.1428,
  'sell_datetime': '2022-06-20 17:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.005500000000000005,
  'proft_loss': 40.058266569555826},
 {'ticker': 'RIFUSDT',
  'buy_price': 0.11,
  'buy_datetime': '2022-07-19 02:00:00',
  'quantity': 9090.90909090909,
  'sell_price': 0.0941,
  'sell_datetime': '2022-07-19 06:00:00',
  'days_to_gain_loss': 0,
  'price_diff': -0.015899999999999997,
  'proft_loss': -144.5454545454545},
 {'ticker': 'PERPUSDT',
  'buy_price': 0.651,
  'buy_datetime': '2022-06-22 03:00:00',
  'quantity': 1536.0983102918585,
  'sell_price': 0.652,
  'sell_datetime': '2022-06-22 07:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.0010000000000000009,
  'proft_loss': 1.5360983102918908},
 {'ticker': 'PERPUSDT',
  'buy_price': 0.78,
  'buy_datetime': '2022-06-22 13:00:00',
  'quantity': 1282.051282051282,
  'sell_price': 0.73

In [16]:
len(trade_list)

9

In [17]:
df = pd.DataFrame.from_dict(trade_list)

In [18]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] > 0])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,FLMUSDT,0.1373,2022-06-20 13:00:00,7283.321194,0.14280,2022-06-20 17:00:00,0,0.00550,40.058267
2,PERPUSDT,0.6510,2022-06-22 03:00:00,1536.098310,0.65200,2022-06-22 07:00:00,0,0.00100,1.536098
4,PERPUSDT,0.8860,2022-07-17 07:00:00,1128.668172,0.97460,2022-07-17 10:00:00,0,0.08860,100.000000
7,TLMUSDT,0.0275,2022-07-06 22:00:00,36363.636364,0.03025,2022-07-06 23:00:00,0,0.00275,100.000000


In [19]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] < 0])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
1,RIFUSDT,0.11000,2022-07-19 02:00:00,9090.909091,0.09410,2022-07-19 06:00:00,0,-0.01590,-144.545455
3,PERPUSDT,0.78000,2022-06-22 13:00:00,1282.051282,0.73900,2022-06-22 17:00:00,0,-0.04100,-52.564103
5,SUPERUSDT,0.13570,2022-07-01 12:00:00,7369.196758,0.12690,2022-07-01 16:00:00,0,-0.00880,-64.848931
6,SUPERUSDT,0.13710,2022-07-07 07:00:00,7293.946025,0.13240,2022-07-07 11:00:00,0,-0.00470,-34.281546
8,BETAUSDT,0.13892,2022-07-14 22:00:00,7198.387561,0.11699,2022-07-15 02:00:00,0,-0.02193,-157.860639


In [20]:
with pd.option_context('display.max_rows', None):
    display(df[pd.isna(df['sell_price'])])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss


In [21]:
df['proft_loss'].sum()

-212.50630922955202

In [22]:
with pd.option_context('display.max_rows', None):
    display(df[np.invert(pd.isna(df['sell_price']))])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,FLMUSDT,0.13730,2022-06-20 13:00:00,7283.321194,0.14280,2022-06-20 17:00:00,0,0.00550,40.058267
1,RIFUSDT,0.11000,2022-07-19 02:00:00,9090.909091,0.09410,2022-07-19 06:00:00,0,-0.01590,-144.545455
2,PERPUSDT,0.65100,2022-06-22 03:00:00,1536.098310,0.65200,2022-06-22 07:00:00,0,0.00100,1.536098
3,PERPUSDT,0.78000,2022-06-22 13:00:00,1282.051282,0.73900,2022-06-22 17:00:00,0,-0.04100,-52.564103
4,PERPUSDT,0.88600,2022-07-17 07:00:00,1128.668172,0.97460,2022-07-17 10:00:00,0,0.08860,100.000000
5,SUPERUSDT,0.13570,2022-07-01 12:00:00,7369.196758,0.12690,2022-07-01 16:00:00,0,-0.00880,-64.848931
6,SUPERUSDT,0.13710,2022-07-07 07:00:00,7293.946025,0.13240,2022-07-07 11:00:00,0,-0.00470,-34.281546
7,TLMUSDT,0.02750,2022-07-06 22:00:00,36363.636364,0.03025,2022-07-06 23:00:00,0,0.00275,100.000000
8,BETAUSDT,0.13892,2022-07-14 22:00:00,7198.387561,0.11699,2022-07-15 02:00:00,0,-0.02193,-157.860639
